In [1]:
from pyforest import *

In [2]:
lazy_imports()

['from sklearn.ensemble import GradientBoostingClassifier',
 'import pydot',
 'from sklearn.ensemble import RandomForestClassifier',
 'from sklearn.feature_extraction.text import CountVectorizer',
 'from sklearn.model_selection import StratifiedKFold',
 'import xgboost as xgb',
 'import pickle',
 'import matplotlib.pyplot as plt',
 'import tqdm',
 'from fbprophet import Prophet',
 'import glob',
 'from sklearn.preprocessing import OneHotEncoder',
 'import seaborn as sns',
 'import plotly.graph_objs as go',
 'import plotly.express as px',
 'from sklearn.feature_extraction.text import TfidfVectorizer',
 'from statsmodels.tsa.arima_model import ARIMA',
 'from sklearn.model_selection import train_test_split',
 'from PIL import Image',
 'from sklearn.linear_model import Ridge',
 'from sklearn.ensemble import RandomForestRegressor',
 'import awswrangler as wr',
 'import cv2',
 'from sklearn.linear_model import RidgeCV',
 'from sklearn.impute import SimpleImputer',
 'import gensim',
 'import 

In [3]:
df_train = pd.read_csv('../../dataset/bigmart_train.csv')
df_test = pd.read_csv('../../dataset/bigmart_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
df_test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [6]:
print(df_train.shape)
print(df_test.shape)

(8523, 12)
(5681, 11)


In [7]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Item_Weight,7060.0,12.857645,4.643456,4.555,8.773750,12.600000,16.850000,21.350000
Item_Visibility,8523.0,0.066132,0.051598,0.000,0.026989,0.053931,0.094585,0.328391
Item_MRP,8523.0,140.992782,62.275067,31.290,93.826500,143.012800,185.643700,266.888400
Outlet_Establishment_Year,8523.0,1997.831867,8.371760,1985.000,1987.000000,1999.000000,2004.000000,2009.000000
Item_Outlet_Sales,8523.0,2181.288914,1706.499616,33.290,834.247400,1794.331000,3101.296400,13086.964800


In [8]:
df_train.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [9]:
df_train['source'] = 'train'
df_test['source'] = 'test'

df = pd.concat([df_train, df_test], ignore_index=True)

<IPython.core.display.Javascript object>

In [10]:
df.tail()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
14199,FDB58,10.5,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1,NaN,test
14200,FDD47,7.6,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2,NaN,test
14201,NCO17,10.0,Low Fat,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,Tier 2,Supermarket Type1,NaN,test
14202,FDJ26,15.3,Regular,0.000000,Canned,214.6218,OUT017,2007,NaN,Tier 2,Supermarket Type1,NaN,test
14203,FDU37,9.5,Regular,0.104720,Canned,79.7960,OUT045,2002,NaN,Tier 2,Supermarket Type1,NaN,test


In [11]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [ ]:
for i in df_train.describe().columns:
    sns.displot(df_train[i].dropna(),kde=True)
    plt.show()

In [ ]:
for i in df_train.describe().columns:
    sns.boxplot(df_train[i].dropna(), orient="h")
    plt.xlabel(df_train[i].name)
    plt.show()

In [ ]:
df_train['Item_Type'].value_counts()

In [ ]:
df_train.dtypes

In [ ]:
plt.figure(figsize=(17, 8))
sns.countplot(x=df_train['Item_Type'])
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.countplot(x=df_train['Outlet_Size'])
plt.show()

In [ ]:
df_train['Outlet_Size'].value_counts()

In [ ]:
plt.figure(figsize=(12, 7))
sns.scatterplot(x='Item_Weight', y='Item_Outlet_Sales', hue='Item_Type', size='Item_Weight', data=df_train)
plt.xlabel('Item Visibility')
plt.ylabel('Item Outlet Sales')
plt.show()

In [ ]:
pt = df_train.pivot_table(index='Item_Fat_Content', values='Item_Outlet_Sales', aggfunc=np.median)
pt.plot(kind='bar', color='b', figsize=(12, 7))
plt.xlabel('Item_Fat_Content')
plt.ylabel('Item_Outlet_Sales')
plt.xticks(rotation=0)
plt.show()

In [ ]:
df_train['Item_Fat_Content'].value_counts()

In [ ]:
df_train['Item_Fat_Content'] = df_train['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'})

In [ ]:
df_train['Item_Fat_Content'].value_counts()

In [ ]:
pt = df_train.pivot_table(index='Item_Fat_Content', values='Item_Outlet_Sales', aggfunc=np.median)
pt.plot(kind='bar', color='b', figsize=(12, 7))
plt.xlabel('Item_Fat_Content')
plt.ylabel('Item_Outlet_Sales')
plt.xticks(rotation=0)
plt.show()

In [ ]:
df_train.corr(numeric_only=True)